#Import OpenAI SDK

In [1]:
# Install the openai-agents library
!pip install -Uq openai-agents
!pip install -Uq openai-agents pydantic

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.1/40.1 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.6/130.6 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.3/129.3 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.9/150.9 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 3.6 MB/s eta 0:00:00


#Import Asyncio to run asynchronus functions

In [2]:
# Import the nest_asyncio library
import nest_asyncio
# Apply nest_asyncio to allow asyncio to run in a nested environment (like Colab)
nest_asyncio.apply()

#Set Gemini API key as Default API reference instaed of Open API key and Setting Up Configurations

In [5]:
# Import necessary libraries
import os
from agents.run import RunConfig
from google.colab import userdata
from pydantic import BaseModel

from agents import (
    Agent,
    GuardrailFunctionOutput,
    InputGuardrailTripwireTriggered,
    OutputGuardrailTripwireTriggered,
    RunContextWrapper,
    Runner,
    TResponseInputItem,
    input_guardrail,
    output_guardrail,
    set_default_openai_api,
    AsyncOpenAI,
    OpenAIChatCompletionsModel,
    set_default_openai_client,
    set_tracing_disabled,
)

# Get the Gemini API key from Colab user data secrets
gemini_api_key = userdata.get('GEMINI_API_KEY')

# Disable tracing for the agents library
set_tracing_disabled(True)
# Set the default OpenAI API to chat completions - this is needed to use the AsyncOpenAI client with Gemini
set_default_openai_api("chat_completions")

# Create an AsyncOpenAI client with the Gemini API key and base URL
external_client = AsyncOpenAI(
    api_key=gemini_api_key,
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/",
)
# Set the default OpenAI client to the external client
set_default_openai_client(external_client)

# Create an OpenAIChatCompletionsModel instance using the Gemini 2.0 Flash model
model = OpenAIChatCompletionsModel(
    model="gemini-2.0-flash",
    openai_client= external_client
)

# Configure the run with the specified model, model provider, and tracing disabled
config = RunConfig(
    model=model,
    model_provider=external_client,
    tracing_disabled=True
)

#Guardrails
##1. Input Guardrail

In [7]:
# Define the output type for the guardrail agent
class MathHomeworkOutput(BaseModel):
    is_math_homework: bool
    reasoning: str

# Create an agent specifically for checking if the input is math homework
guardrail_agent = Agent(
    name="Guardrail check",
    instructions="Check if the user is asking you to do their math homework.",
    output_type=MathHomeworkOutput,
)

# Define the input guardrail function
@input_guardrail
async def math_guardrail(
    ctx: RunContextWrapper[None], agent: Agent, input: str | list[TResponseInputItem]
) -> GuardrailFunctionOutput:
    # Run the guardrail agent with the user input to check for math homework
    result = await Runner.run(guardrail_agent, input, context=ctx.context, run_config = config)

    # Return the guardrail output, triggering the tripwire if it's math homework
    return GuardrailFunctionOutput(
        output_info=result.final_output,
        # tripwire_triggered=False #result.final_output.is_math_homework, # This line seems to be commented out, keeping the next line active
        tripwire_triggered=result.final_output.is_math_homework,
    )

# Create the main agent with the math homework input guardrail
agent = Agent(
    name="Customer support agent",
    instructions="You are a customer support agent. You help customers with their questions.",
    input_guardrails=[math_guardrail],
)

# This block tests if the guardrail trips when the input is math homework
try:
    result = await Runner.run(agent, "Hello, can you help me solve for x: 2x + 3 = 11?", run_config = config)
    print("Guardrail didn't trip - this is unexpected")
    print(result.final_output)

except InputGuardrailTripwireTriggered:
    print("Test 1 successful")
    print("Math homework guardrail tripped")

# This block tests if the agent runs normally when the input is not math homework
try:
    result = await Runner.run(agent, "Hello", run_config = config)
    print("Test 2 successful")
    print(result.final_output)

except InputGuardrailTripwireTriggered:
    print("Math homework guardrail tripped")

Test 1 successful
Math homework guardrail tripped
Test 2 successful
Hi there! How can I help you today?



##2. Output Guardrail

In [9]:
# Define the output type for the main agent's response
class MessageOutput(BaseModel):
    response: str

# Define the output type for the output guardrail agent
class MathOutput(BaseModel):
    is_math: bool
    reasoning: str

# Create an agent specifically for checking if the output includes math
guardrail_agent2 = Agent(
    name="Guardrail check",
    instructions="Check if the output includes any math.",
    output_type=MathOutput,
)

# Define the output guardrail function
@output_guardrail
async def math_guardrail2(
    ctx: RunContextWrapper, agent: Agent, output: MessageOutput
) -> GuardrailFunctionOutput:
    # Run the guardrail agent with the agent's output to check for math
    result = await Runner.run(guardrail_agent2, output.response, context=ctx.context, run_config = config)

    # Return the guardrail output, triggering the tripwire if the output contains math
    return GuardrailFunctionOutput(
        output_info=result.final_output,
        tripwire_triggered=result.final_output.is_math,
    )

# Create the main agent with the math output guardrail and a specified output type
agent2 = Agent(
    name="Customer support agent",
    instructions="You are a customer support agent. You help customers with their questions.",
    output_guardrails=[math_guardrail2],
    output_type=MessageOutput,
)

# This block tests if the output guardrail trips when the output contains math
try:
    await Runner.run(agent2, "Hello, can you help me solve for x: 2x + 3 = 11?", run_config = config)
    print("Guardrail didn't trip - this is unexpected")

except OutputGuardrailTripwireTriggered:
    print("Test 1 successful")
    print("Math output guardrail tripped")

# This block tests if the agent runs normally when the output does not contain math
try:
    result = await Runner.run(agent, "Hello", run_config = config)
    print("Test 2 successful")
    print(result.final_output)

except InputGuardrailTripwireTriggered:
    print("Math homework guardrail tripped")

Test 1 successful
Math output guardrail tripped
Test 2 successful
Hi there! How can I help you today?

